# Experimentos Finales Modelos no RN

In [1]:
!pip install umap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install ml_metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np

import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import datetime
from dateutil.parser import parse
from sklearn.tree import  DecisionTreeClassifier
import plotly.graph_objects as go
import umap
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import ml_metrics
import pickle
from sklearn.metrics import accuracy_score
import joblib
import json


In [4]:
Data=pd.read_csv("cleaned_01June2022.csv").drop(columns=["Unnamed: 0"]).fillna("")
Data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,pref_B-B,pref_D-E,pref_C-D,pref_A-A,Duracion_Campaña,B,C,lec_B-B,lec_E-E,lec_D-E,...,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,Target
0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,5.0,0.0,0.0,...,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,
1,0.0,0.0,0.0,0.0,1.0,2.0,9.0,14.0,1.0,1.0,...,R9,0,B,Mayor a 10 años,37.0,5.0,D,E,P164,D-E C-D E-E B-B
2,0.0,0.0,0.0,0.0,1.0,2.0,2.0,5.0,0.0,0.0,...,R9,1,B,Mayor a 10 años,19.0,4.0,B,D,P114,E-E A-A B-B
3,0.0,0.0,0.0,0.0,1.0,6.0,6.0,8.0,0.0,1.0,...,R4,0,A,Mayor a 10 años,91.0,13.0,D,B,P85,E-E B-B D-E
4,0.0,0.0,0.0,0.0,1.0,2.0,3.0,1.0,0.0,0.0,...,R5,0,A,Mayor a 10 años,34.0,5.0,B,C,P143,D-E A-A


In [5]:
# datos númericos
numerical=list(Data.select_dtypes(exclude=["category","object"]).columns)
numerical.remove("Sexo")
numerical.remove('Recibe_sueldo_en_cuenta')


# datos categorical
categorical=list(Data.select_dtypes(include=["category","object"]).columns)+['Sexo','Recibe_sueldo_en_cuenta']
categorical.remove("Target")

In [6]:
for i in categorical:
    Data[i]=Data[i].astype(str)

In [7]:
# Select 1 - Only Gini, corte visual
select_variables_1=['E-F',
 'C-C',
 'D-F',
 'A-K',
 'num_comunicaciones',
 'Length',
 'pref_A-A',
 'Positivo',
 'Recency',
 'A-G',
 'Monto',
 'Periodicity',
 'Frequency',
 'lec_B-B',
 'G-K']

In [8]:
# Select 2 - Criterio Corr Kill
select_variables_2=['E-F',
 'C-C',
 'D-F',
 'A-K',
 'pref_A-A',
 'num_comunicaciones',
 'Length',
 'Sin Info',
 'F-I',
 'Monto',
 'B',
 'lec_B-B',
 'Comuna',
 'Edad',
 'Ciudad',
 'Profesion',
 'Renta',
 'F-H',
 'lec_A-A',
 'Principalidad',
 'A-L',
 'lec_C-D',
 'Estado_civil']

# Train, Test Split

In [9]:
X=Data.drop(columns=["Target"]).copy()
X.head()

,pref_B-B,pref_D-E,pref_C-D,pref_A-A,Duracion_Campaña,B,C,lec_B-B,lec_E-E,lec_D-E,...,Sexo,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion
0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,5.0,0.0,0.0,...,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164
1,0.0,0.0,0.0,0.0,1.0,2.0,9.0,14.0,1.0,1.0,...,1,R9,0,B,Mayor a 10 años,37.0,5.0,D,E,P164
2,0.0,0.0,0.0,0.0,1.0,2.0,2.0,5.0,0.0,0.0,...,1,R9,1,B,Mayor a 10 años,19.0,4.0,B,D,P114
3,0.0,0.0,0.0,0.0,1.0,6.0,6.0,8.0,0.0,1.0,...,1,R4,0,A,Mayor a 10 años,91.0,13.0,D,B,P85
4,0.0,0.0,0.0,0.0,1.0,2.0,3.0,1.0,0.0,0.0,...,1,R5,0,A,Mayor a 10 años,34.0,5.0,B,C,P143


In [10]:
Y=Data["Target"]
Y.head()

0                   
1    D-E C-D E-E B-B
2        E-E A-A B-B
3        E-E B-B D-E
4            D-E A-A
Name: Target, dtype: object

In [11]:
tf = open("myDictionary.json", "r")
dic_target = json.load(tf)
print(dic_target)

{'': 0, 'A-A': 36, 'A-A B-B': 15, 'A-A B-B D-E': 43, 'A-A C-D': 73, 'A-A D-E': 44, 'A-A D-E B-B': 50, 'A-A E-E': 39, 'B-B': 18, 'B-B A-A': 35, 'B-B A-A D-E': 42, 'B-B C-D': 59, 'B-B C-D D-E': 37, 'B-B C-D D-E E-E': 64, 'B-B C-D E-E': 62, 'B-B D-E': 13, 'B-B D-E A-A': 71, 'B-B D-E C-D': 60, 'B-B D-E E-E': 29, 'B-B E-E': 9, 'B-B E-E C-D': 65, 'B-B E-E D-E': 34, 'C-D': 21, 'C-D A-A': 54, 'C-D A-A B-B': 25, 'C-D B-B': 20, 'C-D B-B A-A': 61, 'C-D B-B D-E': 16, 'C-D B-B D-E E-E': 68, 'C-D B-B E-E': 12, 'C-D B-B E-E D-E': 14, 'C-D D-E': 31, 'C-D D-E B-B': 24, 'C-D D-E B-B E-E': 56, 'C-D D-E E-E': 23, 'C-D D-E E-E B-B': 70, 'C-D E-E': 19, 'C-D E-E B-B': 32, 'C-D E-E B-B D-E': 38, 'C-D E-E D-E': 28, 'C-D E-E D-E B-B': 33, 'D-E': 6, 'D-E A-A': 4, 'D-E A-A B-B': 41, 'D-E B-B': 7, 'D-E B-B A-A': 8, 'D-E B-B C-D': 63, 'D-E B-B E-E': 46, 'D-E C-D': 45, 'D-E C-D B-B': 40, 'D-E C-D B-B E-E': 66, 'D-E C-D E-E': 67, 'D-E C-D E-E B-B': 1, 'D-E E-E': 22, 'D-E E-E A-A': 72, 'D-E E-E B-B': 10, 'D-E E-E C-D'

In [12]:
Y.replace(dic_target,inplace=True)

In [13]:
# Generación de conjuntos de train y test
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, Y, test_size=0.33, 
    shuffle=True,
    
) 

In [14]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

In [15]:
# ROS:
Xtrain, Ytrain=ros.fit_resample(Xtrain,Ytrain)

In [16]:
Xtrain.shape

(608650, 43)

In [17]:
numerical1=list(filter(lambda x:x  in numerical,select_variables_1))
categorical1=list(filter(lambda x:x  in categorical,select_variables_1))

In [18]:
numerical2=list(filter(lambda x:x  in numerical,select_variables_2))
categorical2=list(filter(lambda x:x  in categorical,select_variables_2))

In [19]:
# Preprocesamiento de datos
preprocessing_transformer1 = ColumnTransformer(
    transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore"),categorical1 ),
        ('MinMax', MinMaxScaler(),numerical1),
    ])

In [20]:
# Preprocesamiento de datos
preprocessing_transformer2 = ColumnTransformer(
    transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore"),categorical2 ),
        ('MinMax', MinMaxScaler(),numerical2),
    ])

# Exp 1

In [21]:
pipe_xgb1 = Pipeline(
    [("preprocesamiento", preprocessing_transformer1),
     ("clf", xgb.XGBClassifier(seed=1,  eval_metric='mlogloss',
                               learning_rate= 0.05,n_estimators= 500,
                               tree_method='gpu_hist', gpu_id=0
                               ))]
)


In [22]:
# Fit
pipe_xgb1.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  []),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['E-F', 'C-C', 'D-F', 'A-K',
                                                   'num_comunicaciones',
                                                   'Length', 'pref_A-A',
                                                   'Positivo', 'Recency', 'A-G',
                                                   'Monto', 'Periodicity',
                                                   'Frequency', 'lec_B-B',
                                                   'G-K'])])),
                ('clf',
                 XGBClassifier(eval_metric='mlogloss', gpu_id=0,
                               learning_rate=0.05, n_estimators=500

In [23]:
inv_map = {v: k for k, v in dic_target.items()}

In [24]:
# Generación de predicciones
Ypred =pd.DataFrame(pipe_xgb1.predict(Xtest)).replace(inv_map)[0]

In [25]:
Ytest.replace(inv_map,inplace=True)

In [26]:
accuracy_score(Ypred, Ytest)

0.2867803837953092

In [27]:
Ypred_map=Ypred.apply( lambda x: x.split(" "))
Ytest_map=Ytest.apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.7153147950722576

In [28]:
pipe_xgb1[1]

XGBClassifier(eval_metric='mlogloss', gpu_id=0, learning_rate=0.05,
              n_estimators=500, objective='multi:softprob', seed=1,
              tree_method='gpu_hist')

In [29]:
# save to JSON
pipe_xgb1[1].save_model("./model_xgb1.bin")
